# Importing Modules and Loading Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataCleaner import preProcess
from helper_fns import lexicon_calculate
from Sequencer import Sequencer
import nltk
import json

%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
tweets_df = pd.read_csv('Datasets/tweets_sns2.csv')
tweets_df.head()

,user_name,user_location,text,likes,date
0,klbe1997,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
1,SaMar2555_,الوادي الجديد,متعتك عندي بتحويل فودافون كاش سكس فون و كاميرا...,2,2022-08-28
2,dyna86889772,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
3,hcintma,الوادي الجديد,شعري لونه orange وانتوا؟!,0,2022-08-28
4,alikreek564,الوادي الجديد,شعري لونه orange وانتوا؟!,1,2022-08-28


In [61]:
tweets_df.shape

(6876, 5)

# Cleaning and Preprocessing the Data

In [62]:
keep_regex = "[Vv]odafone|VODAFONE|فودافون|[Ee]tisalat|ETISALAT|اتصالات|[Oo]range|ORANGE|اورانج|موبينيل|إتصالات|أورانج"
remove_regex = "لون"

In [63]:
drop_indeces = tweets_df[(tweets_df['text'].str.contains(keep_regex)==False) | (tweets_df['text'].str.contains(remove_regex)==True)].index

In [64]:
tweets_df.drop(drop_indeces, inplace=True)
tweets_df.shape

(6348, 5)

In [65]:
tweets_df.iloc[179, 2]

'اعلانات اتصالات وحشاني 😂😂'

In [66]:
duplicate_indeces = tweets_df[tweets_df.duplicated('text')].index

In [67]:
tweets_df.drop(duplicate_indeces, inplace=True)
tweets_df.shape

(2817, 5)

In [50]:
tweets_df.to_csv('Datasets/tweets_filtered.csv')

In [51]:
tweets_df = tweets_df.rename(columns={"text":"tweet"})

In [5]:
from dataCleaner import clean_arabic_text, farasa_stemmer, normalize, qalsadi_lemmatizer, remove_diacritics, remove_links, remove_longation, remove_tag_persons, replace_emojies, snow_ball_stemmer, stem_docs

tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : remove_tag_persons(tweet))
tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : remove_links(tweet))
# tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : remove_diacritics(tweet))
tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : replace_emojies(tweet))
tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : clean_arabic_text(tweet))
tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : normalize(tweet))
tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : remove_longation(tweet))
# tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : stem_docs(tweet))
# tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : farasa_stemmer(tweet))
# tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : qalsadi_lemmatizer(tweet))
tweets_df['tweet'] = tweets_df.tweet.map(lambda tweet : snow_ball_stemmer(tweet))

In [14]:
tweets_df.to_csv("Preprocessing Trials/train_rtp_rl_re_cat_n_rl_snowball.csv")

# Comparing Corpuses

In [52]:
tweets= tweets_df['tweet'].copy()

In [53]:
df2 = pd.read_csv('Datasets/tweets_train.csv')
df1 = pd.read_csv('Datasets/NU_EG_Twitter_corpus_train.csv')
df2.rename(columns={"text":"tweet"},inplace=True)
df = df2.append(df1,ignore_index = True)
preProcess(df)
tweets2 = df['tweet'].copy()

C:\Users\youss\AppData\Local\Temp\ipykernel_21020\1748729805.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df2.append(df1,ignore_index = True)


In [54]:
tokens1 = set(nltk.tokenize.wordpunct_tokenize(' '.join(tweets.to_numpy().flatten())))
tokens2 = set(nltk.tokenize.wordpunct_tokenize(' '.join(tweets2.to_numpy().flatten())))

In [55]:
print(len(set(tokens1)), len(set(tokens2)))

7686 25581


In [56]:
print(len(set(tokens1).difference(set(tokens2))))

3202


In [57]:
def tweet_filter(tweet: str, bucket: set[str]) -> bool:
    tokens = set(nltk.tokenize.wordpunct_tokenize(tweet))
    inclusion_ratio =  len(tokens.intersection(bucket)) / len(tokens)
    return inclusion_ratio >= 0.6


filtered_tweets = tweets.to_numpy()[[tweet_filter(x, tokens2) for x in tweets.to_numpy()]]

In [58]:
print(len(filtered_tweets))

2637


# Predicting and Analysis

In [ ]:
import pickle

with open('Models/nb_lexicon_model.sav', mode='rb') as f:
    nb_lex_count_model = pickle.load(f)